In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SGMD Khayet did.xlsx to SGMD Khayet did.xlsx


In [ ]:
df = pd.read_excel("SGMD Khayet did.xlsx")

In [ ]:
df

,Trial,Type,"Tf,in (°C)",Ua (m/s),Uw (m/s),J (kg m−2 h−1)
0,1,Training,68,1.932,0.200,3.9816
1,2,Training,54,1.932,0.200,2.2896
2,3,Training,68,0.966,0.200,2.3400
3,4,Training,54,0.966,0.200,1.6524
4,5,Training,68,1.932,0.140,3.4884
5,6,Training,54,1.932,0.140,2.6640
6,7,Training,68,0.966,0.140,2.0952
7,8,Training,54,0.966,0.140,1.1736
8,9,Training,70,1.449,0.170,3.6576
9,10,Training,52,1.449,0.170,1.7964


In [ ]:
#  DataFrame is named `df`
# Extract the target column (assuming the last column is the target)
target_column = df.columns[-1]  # Get the name of the last column
flux_values = df[target_column]

# Calculate the minimum, maximum, and range values
min_flux = flux_values.min()
max_flux = flux_values.max()
range_flux = max_flux - min_flux

# Print the results
print("Minimum Flux:", min_flux)
print("Maximum Flux:", max_flux)
print("Range of Flux:", range_flux)


Minimum Flux: 1.1736
Maximum Flux: 3.9816000000000003
Range of Flux: 2.8080000000000003


In [ ]:
print(df.dtypes)


Trial               int64
Type               object
Tf,in (°C)          int64
Ua (m/s)          float64
Uw (m/s)          float64
J (kg m−2 h−1)    float64
dtype: object


In [ ]:
X = df.iloc[: ,2: -1]
X= X.to_numpy()
print(X.shape)


y = df.iloc[: , -1]
y = y.to_numpy()
print(y.shape)

(53, 3)
(53,)


In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining the K-Fold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

mse_list = []
r2_list = []
train_mse_list = []

# K-Fold cross-validation
for train_indices, dev_indices in kf.split(X_train):
    X_train_fold, X_dev_fold = X_train[train_indices], X_train[dev_indices]
    y_train_fold, y_dev_fold = y_train[train_indices], y_train[dev_indices]

    scaler = StandardScaler()
    X_train_fold_scaled = scaler.fit_transform(X_train_fold)
    X_dev_fold_scaled = scaler.transform(X_dev_fold)

    model = Sequential([
        Dense(units=32, activation='relu', input_shape=(X_train_fold_scaled.shape[1],)),
        Dense(units=16, activation='relu'),
        Dense(units=8, activation='relu'),
        Dense(units=1, activation='linear'),
    ])
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.001))
    model.fit(X_train_fold_scaled, y_train_fold, epochs=1000, verbose=0)

    # Calculate predictions and MSE for the development fold
    y_dev_fold_prediction = model.predict(X_dev_fold_scaled)
    mse = mean_squared_error(y_dev_fold, y_dev_fold_prediction)
    r2 = r2_score(y_dev_fold, y_dev_fold_prediction)
    mse_list.append(mse)
    r2_list.append(r2)

    # Calculate predictions and MSE for the training fold
    y_train_fold_prediction = model.predict(X_train_fold_scaled)
    train_mse = mean_squared_error(y_train_fold, y_train_fold_prediction)
    train_mse_list.append(train_mse)

mse_average = np.mean(mse_list)
r2_average = np.mean(r2_list)
train_mse_average = np.mean(train_mse_list)

print(f"MSE for all development folds: {mse_list}, average: {mse_average:.4f}")
print(f"R2 for all development folds: {r2_list}, average: {r2_average:.4f}")
print(f"MSE for all training folds: {train_mse_list}, average: {train_mse_average:.4f}")



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
MSE for all development folds: [0.22682539734962953, 0.19159051814079733, 0.2152268563957233, 0.0914419397277655, 0.08462200797556585], average: 0.1619
R2 for all development folds: [0.5641567158665519, 0.7455702083814101, 0.5013525106656191, 0.7067741730580884, 0.709507349696223], average: 0.6455
MSE for all training folds: [0.010895195673940344, 0.010125937685489365, 0.007764533282959938, 0.019599982418175566, 0.0034972819068857853], average: 0.0104


In [ ]:
# Scaling the entire training set and the test set
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training the final model on the entire training set
model = Sequential([
    Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(units=32, activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(units=1, activation='linear'),
])

model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.001))
model.fit(X_train_scaled, y_train, epochs=800, verbose=0)

# Evaluating the final model on the test set
y_test_prediction = model.predict(X_test_scaled)
mse_test = mean_squared_error(y_test, y_test_prediction)
r2_test = r2_score(y_test, y_test_prediction)
print(f'mse for the test is: {mse_test:.3f}')
print(f'R^2 for the test is: {r2_test:.3f}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
mse for the test is: 0.099
R^2 for the test is: 0.726


In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# Assuming you have the following variables already defined
# y_test: actual target values for the test set
# y_test_pred: predicted values for the test set
# mae_test: mean absolute error calculated previously

# Calculate MAE for the test dataset (if not already done)
mae_test = mean_absolute_error(y_test, y_test_prediction)

# Calculate the mean of the target values for the Relative MAE
mean_target = np.mean(y_test)

# Calculate Relative MAE for the test dataset
relative_mae_test = (mae_test / mean_target) * 100

# Print results
print(f"MAE for test dataset is: {mae_test:.3f}")
print(f"Relative MAE for test dataset is: {relative_mae_test:.2f}%")


MAE for test dataset is: 0.240
Relative MAE for test dataset is: 8.82%


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# Assuming df is your DataFrame and you have correctly extracted X and y
# X and y have been verified as:
# X.shape: (69, 5)
# y.shape: (69,)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (normalize)
scaler = StandardScaler()

# Define the parameter grid for grid search
param_grid = {
    'svr__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
    'svr__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 'scale', 'auto'],
    'svr__epsilon': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'pca__n_components': [2, 3, 4, 5]  # Adding PCA components as part of the grid search
}

# Create a pipeline with PCA and SVR
pipeline = Pipeline([
    ('scaler', scaler),
    ('pca', PCA()),
    ('svr', SVR(kernel='rbf'))
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Train the model
grid_search.fit(X_train, y_train)

# Get the best model with optimized parameters
best_model = grid_search.best_estimator_

# Make predictions on training and testing sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Evaluate model performance on training set
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Evaluate model performance on testing set
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Print the results
print("Best Parameters:", grid_search.best_params_)
print("Train Mean Squared Error:", train_mse)
print("Train R2 Score:", train_r2)
print("Test Mean Squared Error:", test_mse)
print("Test R2 Score:", test_r2)

Best Parameters: {'pca__n_components': 3, 'svr__C': 1000, 'svr__epsilon': 0.1, 'svr__gamma': 0.001}
Train Mean Squared Error: 0.05846048734071905
Train R2 Score: 0.8770598321481851
Test Mean Squared Error: 0.11577460160171014
Test R2 Score: 0.6795121473261996


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
4480 fits failed out of a total of 8960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2240 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
  File "/usr/local/lib/python3.10/dist-packag

In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE for the test dataset
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the MAE result
print(f"mae for test dataset is: {mae_test:.3f}")


mae for test dataset is: 0.223


In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# Assuming you have the following variables already defined
# y_test: actual target values for the test set
# y_test_pred: predicted values for the test set
# mae_test: mean absolute error calculated previously

# Calculate MAE for the test dataset (if not already done)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Calculate the mean of the target values for the Relative MAE
mean_target = np.mean(y_test)

# Calculate Relative MAE for the test dataset
relative_mae_test = (mae_test / mean_target) * 100

# Print results
print(f"MAE for test dataset is: {mae_test:.3f}")
print(f"Relative MAE for test dataset is: {relative_mae_test:.2f}%")


MAE for test dataset is: 0.223
Relative MAE for test dataset is: 8.19%


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizing features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Set the parameters for XGBoost
params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.1,
    'max_depth': 3,
    'colsample_bytree': 0.8,
    'alpha': 1,  # L1 regularization
    'lambda': 1.5,  # L2 regularization
    'subsample': 0.8,
    'seed': 42
}

# Create the model
xgb_model = XGBRegressor(**params)

# Training the model
xgb_model.fit(X_train_scaled, y_train)

# Making predictions
y_train_pred = xgb_model.predict(X_train_scaled)
y_test_pred = xgb_model.predict(X_test_scaled)

# Evaluation using MSE, R-squared, and MAE
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# Calculate Relative MAE (rMAE)
relative_mae = (test_mae / np.mean(np.abs(y_test))) * 100

# Print the evaluation metrics
print(f"Training Mean Squared Error (MSE): {train_mse:.3f}")
print(f"Training R-squared (R^2): {train_r2:.3f}")
print(f"Test Mean Squared Error (MSE): {test_mse:.3f}")
print(f"Test R-squared (R^2): {test_r2:.3f}")
print(f"Test MAE: {test_mae:.3f}")
print(f"Relative MAE: {relative_mae:.2f}%")


Training Mean Squared Error (MSE): 0.039
Training R-squared (R^2): 0.918
Test Mean Squared Error (MSE): 0.107
Test R-squared (R^2): 0.705
Test MAE: 0.239
Relative MAE: 8.79%
